In [1]:
import mne

import ssvepy

from autoreject import Ransac

import numpy as np
import matplotlib.pyplot as plt

import os
import glob

from tqdm import tqdm_notebook as tqdm
from ipywidgets import interact

mne.utils.set_log_level('WARNING')

/Library/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
if os.path.isdir('/Users/jan/Documents/eeg-data/cancan-saturation/'):
    datafolder = '/Users/jan/Documents/eeg-data/cancan-saturation/'

files = glob.glob(datafolder + '*saturation*.vhdr')

In [3]:
raws = [mne.io.read_raw_brainvision(file, event_id={'DCC': 199, 'actiCAP Data On': 200},
                                    montage=mne.channels.read_montage('standard_1020'))
        for file in files]

In [4]:
epochs = [[mne.Epochs(raw,
                      mne.find_events(raw),
                      event_id=event,
                      tmin=0, tmax=10,
                      picks=mne.pick_types(raw.info, eeg=True))
           for event in tqdm([16, 32, 64, 100], desc='Events')]
          for raw in tqdm(raws, desc='Subject')]

In [5]:
for subject in tqdm(epochs):
    for epoch in subject:
        epoch.load_data()
        epoch.resample(256)


In [6]:
%%capture
# Clean the data using autoreject's ransac
cleaners = [Ransac(verbose='tqdm_notebook') for raw in raws]

cleanepochs = [[cleaner.fit_transform(epoch) for epoch in epochlist]
               for epochlist, cleaner in tqdm(zip(epochs, cleaners), desc='Subjects')]

In [7]:
ssveps = [[ssvepy.Ssvep(epoch, 5.0, fmin=2, fmax=30) for epoch in epochlist]
           for epochlist in tqdm(cleanepochs)]

/Users/jan/Dropbox/Documents/side-projects/ssvepy/ssvepy/ssvepyepochs.py:123: RuntimeWarning: invalid value encountered in true_divide
  self.psd[..., noiseband].mean(axis=-1))


In [40]:
occipital_indices = [raws[0].ch_names.index(ch)
                     for ch in ['Oz', 'O1', 'O2', 'POz']]

# Aggregate the amp from the occ. electrode with max SNR
maxoccamps = np.zeros((len(ssveps), len(ssveps[0])))
for subject, ssveplist in enumerate(ssveps):
    
    for condition, ssvep in enumerate(ssveplist):
        maxelec = np.argmax(ssvep.stimulation.snr[:, occipital_indices].mean(axis=0))
        maxelec = occipital_indices[maxelec]
        maxoccamps[subject, condition] = ssvep.stimulation.power[:, maxelec].mean()

# Aggregate the amp from the electrode with overall max SNR
maxamps = np.zeros((len(ssveps), len(ssveps[0])))
for subject, ssveplist in enumerate(ssveps):
    maxelec = np.nanargmax(
        np.stack(
            [ssvep.stimulation.snr.mean(axis=0) for ssvep in ssveplist], axis=-1
        ).mean(axis=-1)
    )
    for condition, ssvep in enumerate(ssveplist):
        maxamps[subject, condition] = ssvep.stimulation.power[:, maxelec].mean()


In [42]:
maxoccamps

array([[  1.62349194e-09,   2.46147812e-09,   1.51611382e-09,
          1.20297545e-09],
       [  5.58086916e-09,   2.32801471e-09,   6.46108464e-09,
          1.11646462e-09]])